In [83]:
import pandas as pd
import numpy as np
import sqlite3

In [84]:
# Connect to SQLite dB
con = sqlite3.connect('/Users/jennadodge/uofo-virt-data-pt-12-2021-u-b/Water_Quality_Analysis/Database/database.sqlite3')

# Create a cursor object
cur = con.cursor()

# SQL query to dF
df = pd.read_sql_query("SELECT * FROM Census_Data INNER JOIN Contaminant_Summary on Census_Data.county_FIPS = Contaminant_Summary.county_FIPS",con)

# close connection
con.close()

In [85]:
# check shape
df.shape

(882, 37)

In [86]:
#Eliminate outliers

In [87]:
df.columns

Index(['county_FIPS', 'Geographic_Area_Name', 'County', 'GEOID',
       'Total_Population', 'White', 'Black', 'Native', 'Asian',
       'Pacific_Islander', 'Other', 'Two_or_more_Races', 'Hispanic',
       'Not_Hispanic', 'Not_White', 'pct_White', 'pct_Black', 'pct_Native',
       'pct_Asian', 'pct_Pacific_Islander', 'pct_Other', 'pct_Not_White',
       'pct_Hispanic', 'pct_Not_Hispanic', 'pct_Two_or_more_Races',
       'Simpson_Race_DI', 'Simpson_Ethnic_DI', 'Shannon_Race_DI',
       'Shannon_Ethnic_DI', 'Gini_Index', 'County_FIPS', 'Num_Contaminants',
       'Sum_Population_Served', 'Sum_ContaminantFactor',
       'Min_Contaminant_Factor', 'Max_Contaminant_Factor',
       'Avg_Contaminant_Factor'],
      dtype='object')

In [88]:
county_info_df = df[['county_FIPS', 'Geographic_Area_Name', 'County', 'GEOID',
       'Total_Population']]

In [89]:
df2 = df[['Simpson_Race_DI', 'Simpson_Ethnic_DI', 'Shannon_Race_DI',
       'Shannon_Ethnic_DI', 'Gini_Index','Sum_ContaminantFactor']]

In [90]:
# # keep only relevant columns - diversity indices, gini index, Sum_ContaminantFactor
# df.drop(columns=['county_FIPS', 'Geographic_Area_Name', 'County', 'GEOID',
#        'Total_Population', 'White', 'Black', 'Native', 'Asian',
#        'Pacific_Islander', 'Other', 'Two_or_more_Races', 'Hispanic',
#        'Not_Hispanic', 'Not_White', 'pct_White', 'pct_Black', 'pct_Native',
#        'pct_Asian', 'pct_Pacific_Islander', 'pct_Other', 'pct_Not_White',
#        'pct_Hispanic', 'pct_Not_Hispanic', 'pct_Two_or_more_Races',
#         'County_FIPS', 'Num_Contaminants',
#        'Sum_Population_Served',
#        'Min_Contaminant_Factor', 'Max_Contaminant_Factor',
#        'Avg_Contaminant_Factor'], inplace=True)

In [91]:
# df2.rename(columns=
# {
#     'Simpson_Race_DI':'Simp_RDI', 
#     'Simpson_Ethnic_DI':'Simp_EDI',
#     'Shannon_Race_DI':'Shan_RDI',
#     'Shannon_Ethnic_DI':'Shan_EDI', 
#     'Gini_Index':'Gini', 
#     'Sum_ContaminantFactor':'Cont_factor'
# },
# inplace=True)

In [92]:
df2.head()

,Simpson_Race_DI,Simpson_Ethnic_DI,Shannon_Race_DI,Shannon_Ethnic_DI,Gini_Index,Sum_ContaminantFactor
0,0.309413,0.217826,0.948124,0.375569,0.4428,4744
1,0.573395,0.141942,1.305087,0.271086,0.4191,4987
2,0.618709,0.197811,1.464587,0.349206,0.4607,15583
3,0.430910,0.200263,1.161943,0.352475,0.4561,5456
4,0.290288,0.237762,0.919484,0.401124,0.4287,42


In [93]:
df2.describe()

,Simpson_Race_DI,Simpson_Ethnic_DI,Shannon_Race_DI,Shannon_Ethnic_DI,Gini_Index,Sum_ContaminantFactor
count,882.000000,882.000000,882.000000,882.000000,882.000000,8.820000e+02
mean,0.280229,0.113581,0.753765,0.211073,0.441717,1.175539e+04
std,0.166876,0.108514,0.373162,0.152220,0.033689,7.711364e+04
min,0.063408,0.011866,0.202798,0.036514,0.315700,0.000000e+00
25%,0.146299,0.040314,0.457942,0.100287,0.417125,1.966500e+03
50%,0.226679,0.069034,0.669062,0.154340,0.439450,4.770000e+03
75%,0.384804,0.142876,0.980501,0.272424,0.463475,1.104950e+04
max,0.768684,0.499984,1.956877,0.693131,0.550900,2.215481e+06


In [94]:
df2.quantile(0.25)

Simpson_Race_DI             0.146299
Simpson_Ethnic_DI           0.040314
Shannon_Race_DI             0.457942
Shannon_Ethnic_DI           0.100287
Gini_Index                  0.417125
Sum_ContaminantFactor    1966.500000
Name: 0.25, dtype: float64

In [95]:
df2[df2['Sum_ContaminantFactor'] < (df2['Sum_ContaminantFactor'].mean() - 1.5*df2['Sum_ContaminantFactor'].std())]

,Simpson_Race_DI,Simpson_Ethnic_DI,Shannon_Race_DI,Shannon_Ethnic_DI,Gini_Index,Sum_ContaminantFactor


In [96]:
df2.std()

Simpson_Race_DI              0.166876
Simpson_Ethnic_DI            0.108514
Shannon_Race_DI              0.373162
Shannon_Ethnic_DI            0.152220
Gini_Index                   0.033689
Sum_ContaminantFactor    77113.640885
dtype: float64

In [97]:
priority_col = ['Contaminant_Score',
                'Simpson_Race_DI_score',
                 'Simpson_Ethnic_DI_score', 
                 'Shannon_Race_DI_score_score',
                 'Shannon_Ethnic_DI_score', 
                 'Gini_Index_score'               
               ]
priority_df = pd.DataFrame(columns=priority_col)
priority_df

,Contaminant_Score,Simpson_Race_DI_score,Simpson_Ethnic_DI_score,Shannon_Race_DI_score_score,Shannon_Ethnic_DI_score,Gini_Index_score


In [98]:
cont = pd.Series(df2.Sum_ContaminantFactor)
SR_DI = pd.Series(df2.Simpson_Race_DI)
SE_DI = pd.Series(df2.Simpson_Ethnic_DI)
SHE_DI = pd.Series(df2.Shannon_Ethnic_DI)
SHR_DI = pd.Series(df2.Shannon_Race_DI)
Gini = pd.Series(df2.Gini_Index)

In [99]:
cont_mean = np.mean(cont)
SR_DI_mean = np.mean(SR_DI)
SE_DI_mean = np.mean(SE_DI)
SHE_DI_mean = np.mean(SHE_DI)
SHR_DI_mean = np.mean(SHR_DI)
Gini_mean = np.mean(Gini)

In [100]:
cont_sd = np.std(cont)
SR_DI_sd = np.std(SR_DI)
SE_DI_sd = np.std(SE_DI)
SHE_DI_sd = np.std(SHE_DI)
SHR_DI_sd = np.std(SHR_DI)
Gini_sd = np.std(Gini)


In [101]:
df3 = df2[['Sum_ContaminantFactor','Simpson_Ethnic_DI', 'Simpson_Race_DI',
       'Shannon_Race_DI', 'Shannon_Ethnic_DI', 'Gini_Index']]

In [102]:
df3.columns


Index(['Sum_ContaminantFactor', 'Simpson_Ethnic_DI', 'Simpson_Race_DI',
       'Shannon_Race_DI', 'Shannon_Ethnic_DI', 'Gini_Index'],
      dtype='object')

### use np.where() twice in succession to create scores

In [103]:
# lower cutoff = mean - standard deviation
Sum_ContaminantFactor_score = np.where(df3.Sum_ContaminantFactor >= (cont_mean - cont_sd), 1, 0)
Simpson_Ethnic_DI_score = np.where(df3.Simpson_Ethnic_DI >= (SE_DI_mean - SE_DI_mean), 1, 0)
Simpson_Race_DI_score = np.where(df3.Simpson_Race_DI >= (SR_DI_mean - SR_DI_sd), 1, 0)
Shannon_Race_DI_score = np.where(df3.Shannon_Race_DI >= (SHR_DI_mean - SHR_DI_sd), 1, 0)
Shannon_Ethnic_DI_score = np.where(df3.Shannon_Ethnic_DI >= (SHE_DI_mean - SHE_DI_sd), 1, 0)
Gini_Index_score = np.where(df3.Gini_Index >= (Gini_mean - 1.5*Gini_sd), 1, 0)

In [104]:
# higher cutoff = mean + 1.5 * standard deviation score = 2, else CURRENT SCORE.
Sum_ContaminantFactor_score = np.where(df3.Sum_ContaminantFactor >= (cont_mean + 1.5*cont_sd), 2, Sum_ContaminantFactor_score)
Simpson_Ethnic_DI_score = np.where(df3.Simpson_Ethnic_DI >= (SE_DI_mean + 1.5*SE_DI_mean), 2, Simpson_Ethnic_DI_score)
Simpson_Race_DI_score = np.where(df3.Simpson_Race_DI >= (SR_DI_mean + 1.5*SR_DI_sd), 2, Simpson_Race_DI_score)
Shannon_Race_DI_score = np.where(df3.Shannon_Race_DI >= (SHR_DI_mean + 1.5*SHR_DI_sd), 2, Shannon_Race_DI_score)
Shannon_Ethnic_DI_score = np.where(df3.Shannon_Ethnic_DI >= (SHE_DI_mean + 1.5*SHE_DI_sd), 2, Shannon_Ethnic_DI_score)
Gini_Index_score = np.where(df3.Gini_Index >= (Gini_mean + 1.5*Gini_sd), 2, Gini_Index_score)

In [105]:
priority_df.Contaminant_Score = Sum_ContaminantFactor_score
priority_df.Simpson_Ethnic_DI_score = Simpson_Ethnic_DI_score
priority_df.Simpson_Race_DI_score = Simpson_Race_DI_score
priority_df.Shannon_Ethnic_DI_score = Shannon_Ethnic_DI_score
priority_df.Shannon_Race_DI_score_score = Shannon_Race_DI_score
priority_df.Gini_Index_score = Gini_Index_score

In [106]:
priority_df

,Contaminant_Score,Simpson_Race_DI_score,Simpson_Ethnic_DI_score,Shannon_Race_DI_score_score,Shannon_Ethnic_DI_score,Gini_Index_score
0,1,1,1,1,1,1
1,1,2,1,1,1,1
2,1,2,1,2,1,1
3,1,1,1,1,1,1
4,1,1,1,1,1,1
...,...,...,...,...,...,...
877,1,2,2,2,2,1
878,1,2,2,2,2,1
879,1,2,2,2,2,1
880,1,2,2,2,2,1


In [107]:
priority_df['Total'] = priority_df.sum(axis=1)

In [108]:
priority_df

,Contaminant_Score,Simpson_Race_DI_score,Simpson_Ethnic_DI_score,Shannon_Race_DI_score_score,Shannon_Ethnic_DI_score,Gini_Index_score,Total
0,1,1,1,1,1,1,6
1,1,2,1,1,1,1,7
2,1,2,1,2,1,1,8
3,1,1,1,1,1,1,6
4,1,1,1,1,1,1,6
...,...,...,...,...,...,...,...
877,1,2,2,2,2,1,10
878,1,2,2,2,2,1,10
879,1,2,2,2,2,1,10
880,1,2,2,2,2,1,10


In [109]:
priority_df.Total.value_counts()


6     541
7      78
5      70
4      65
10     48
8      30
3      26
9      17
11      5
2       1
12      1
Name: Total, dtype: int64

In [110]:
# use np.where() twice again with priority level to assign priority level
priority_level = np.where(priority_df.Total >= 6, 1, 0)
priority_level = np.where(priority_df.Total >= 7, 2, priority_level)

In [111]:
priority_df["Priority"] = priority_level

In [113]:
priority_df.sample(n=20)

,Contaminant_Score,Simpson_Race_DI_score,Simpson_Ethnic_DI_score,Shannon_Race_DI_score_score,Shannon_Ethnic_DI_score,Gini_Index_score,Total,Priority
518,1,1,1,1,1,1,6,1
804,1,1,1,0,1,1,5,0
396,1,1,1,1,1,1,6,1
404,1,1,1,1,1,1,6,1
766,1,2,1,1,1,1,7,2
146,1,1,1,1,1,1,6,1
643,1,1,1,1,1,0,5,0
640,1,1,1,1,1,1,6,1
773,1,1,1,1,1,1,6,1
729,1,1,1,1,1,1,6,1


In [114]:
priority_df['Priority'].value_counts()

1    541
2    179
0    162
Name: Priority, dtype: int64

In [115]:
df['Priority'] = priority_level

In [116]:
priority_df.to_csv('data_with_ternary_priority.csv', index = False)